# LIGO Project
## PyCBC GW Match

In [1]:
# Filter the LIGO .gwf data file using the pyCBC software package
# Based on Alex Nitz's sample code
# Some lines added by Hao Liu to parameterize the program and for outputs

directory = 'Data/' # directory where the data files are located
suffix = '_LOSC_4_V2-1126257414-4096' # suffix of the file name, not including ".gwf"
match_low_frequency_cutoff = 15 # low frequency cutoff for matching, normally chosen to be same as highpass_freq
highpass_freq = 15 # frequency for the initial high pass (Hz)
highpass_edge_cut_left = 16 # time domain left-side cutoff after initial high pass (in second)
highpass_edge_cut_right = 16 # time domain right-side cutoff after initial high pass (in second)
snr_edge_cut = 2 # length of edge-cut at both ends for max-SNR determination. 2016
psd_segment_length = 16 # segment length for calculating the PSD by Welch's averaging

import lal as _lal
from pycbc.frame import read_frame
from pycbc.filter import highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.types.timeseries import load_timeseries
from pycbc.types.frequencyseries import load_frequencyseries
from pycbc.types.timeseries import TimeSeries

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy.signal import tukey
from scipy.interpolate import interp1d

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
print('Start to read and high-pass the data, will cut some edge after that.')

t_slice = 96

strain, strain_i, stilde = {}, {}, {}
for ifo in ['H', 'L']:
    # Read the detector data
    fname = directory + '%s-%s1' % (ifo, ifo) + suffix + '.gwf'
    channel_name = '%s1:LOSC-STRAIN'  % ifo
    strain_i[ifo] = read_frame(fname, channel_name)
    strain[ifo] = strain_i[ifo].time_slice(1126259462-t_slice/2,1126259462+t_slice/2)

    print(' ')
    print('****************************************************')
    print("%s-file is: %s" %(ifo, fname))
    print("%s-file is from %s to %s, duration: %s" 
        %(ifo, strain[ifo].start_time, strain[ifo].end_time, strain[ifo].duration) )

    # save a copy of the unfiltered strain data to disk
    #strain[ifo].numpy().tofile(directory + '%s-%s1' % (ifo, ifo) + suffix + '_unfilt.bin')

    # Initial high pass to remove the strong low-frequency signal
    #strain_i[ifo] = highpass(strain_i[ifo], highpass_freq) 
    strain[ifo] = highpass(strain[ifo], highpass_freq)

    # Edge-cut to remove time corrupted by the high pass filter
    #strain_i[ifo] = strain_i[ifo].crop(highpass_edge_cut_left, highpass_edge_cut_right)
    strain[ifo] = strain[ifo].crop(highpass_edge_cut_left, highpass_edge_cut_right)

    # Print some information
    print("%s-strain after high-pass is now from %s to %s, duration: %s" 
        %(ifo, strain[ifo].start_time, strain[ifo].end_time, strain[ifo].duration) )

    # Also create a frequency domain version of the data
    stilde[ifo] = strain[ifo].to_frequencyseries()

Start to read and high-pass the data, will cut some edge after that.
 
****************************************************
H-file is: Data/H-H1_LOSC_4_V2-1126257414-4096.gwf
H-file is from 1126259414 to 1126259510, duration: 96.0
H-strain after high-pass is now from 1126259430 to 1126259494, duration: 64.0
 
****************************************************
L-file is: Data/L-L1_LOSC_4_V2-1126257414-4096.gwf
L-file is from 1126259414 to 1126259510, duration: 96.0
L-strain after high-pass is now from 1126259430 to 1126259494, duration: 64.0


In [3]:
print('Calculate the PSD from cut strain...')

psd_method = "pycbc"
psds = {}
for ifo in ['H', 'L']:
    
    if psd_method == "pycbc":
        # Calculate the PSD by a Welch-style estimator (with the pyCBC timeseries.psd() method)
        # Then interpolate the PSD to the desired frequency step.
        
        psds[ifo] = interpolate(strain[ifo].psd(int(psd_segment_length)), stilde[ifo].delta_f)
        print(stilde[ifo].delta_f)
        
        # Smooth to the desired corruption length
        psds[ifo] = inverse_spectrum_truncation(psds[ifo], 
                                                psd_segment_length * strain[ifo].sample_rate,
                                                low_frequency_cutoff=highpass_freq,
                                                trunc_method='hann')  
    
    elif psd_method == "losc":
        # Calculate the PSD by a Welch-style estimator (with the mlab.psd() function)
        # Then interpolate, using numpy's interp function, to the desired frequency step.
        
        fs = strain[ifo].sample_rate
        NFFT = 4*fs
        psd_window = np.blackman(NFFT)
        NOVL = NFFT/2
        
        Pxx, freqs = mlab.psd(strain[ifo], Fs = fs, NFFT = NFFT, window=psd_window, noverlap=NOVL)
        datafreqs = stilde[ifo].sample_frequencies
        psd_tmp = np.interp(datafreqs, freqs, Pxx)
        
        
        np.save('%s_TUT_PSD.npy' %(ifo),np.column_stack((datafreqs,psd_tmp)))
    
        psds[ifo] = load_frequencyseries('%s_TUT_PSD.npy' %(ifo))
        
        strain[ifo] = (stilde[ifo] / psds[ifo]**0.5 ).to_timeseries()
        
        strain[ifo].save("CC_test/%s_data.npy" %(ifo))
        stilde[ifo].save("CC_test/%s_freqseries.npy" %(ifo))
    
    # save a copy of the filtered strain data to disk
    #strain[ifo].numpy().tofile(directory + '%s-%s1' % (ifo, ifo) + suffix + '_filt.bin')

print('PSD ready.')

Calculate the PSD from cut strain...


NameError: name 'xrange' is not defined